### Analysis of CHIRPS-GEFS forecasts and observed dry spells
**Note: this notebook needs some cleaning up**

Goal is to see if CHIRPS-GEFS provides a signal for the occurence of a dry spell in the coming 15 days   
We do this by looking at historical CHIRPS-GEFS data, and classifying the occurence of a dry spell according to this forecast per admin 2   
This information is thereafter coupled to historically observed dry spells   

From the first analysis we find that there is not much correlation between forecasted and observed dry spells. However, we need to further understand if the analysis was computed correclty, and if so, what the cause of these differences is. 


Questions
- Defined a dry spell being forecasted if cell with the maximum value that touches the admin2 is forecasted to receive maximum 2mm of rain. Is this a good definition?
- Is it logical to look at the correlation of a dry spell starting at the first date of the forecast, or should we look more broadly at the occurence of a dry spell during the forecast period?
- Any other methodologies to combine binary geospatial timeseries?
- [Given deadline, lets wait but important for future] The size of the CHIRPS-GEFS data is now 23.4GB... Could we do something to make that smaller/not have to save it locally?      
    Climateserv might be an option, includes chirps-gefs data and can select per country and has an API. However, not clear what exactly the data is that you can download (how many days forecast) and API is hard to understand, and not sure if includes latest data. Someone using the API [here](https://github.com/Servir-Mekong/ClimateSERV_CHIRPS-GEFS/blob/master/Get_CHIRPS_GEFSv1/bin/ClimateServ_CHIPS-GEFS.py)

To do:
- Understand why observed and forecasted dry spells don't match
- Get a bit better understanding of the CHIRPS-GEFS methodology
- Make sure the date of the filename is the first date of the forecast --> yes it is
- Adjust rainy season definnitions based on definition of dry spell (max or mean)


Limitations:
- Part of 2020 data missing
- Not all data 2000-2010 has been downloaded
- 

### set general variables and functions

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from rasterstats import zonal_stats
import rasterio
from rasterio.enums import Resampling
import matplotlib
import matplotlib.colors as mcolors
import xarray as xr
import cftime
import math
import rioxarray
from shapely.geometry import mapping
import cartopy.crs as ccrs
import matplotlib as mpl
import datetime
from datetime import timedelta
import re
import seaborn as sns

In [29]:
from pathlib import Path
import sys
import os

path_mod = f"{Path(os.path.dirname(os.path.abspath(''))).parents[1]}/"
# print(path_mod)
sys.path.append(path_mod)
from src.indicators.drought.config import Config
from src.utils_general.utils import download_ftp,download_url
from src.utils_general.raster_manipulation import fix_calendar, invert_latlon, change_longitude_range
from src.utils_general.plotting import plot_raster_boundaries_clip

#### Set config values

In [4]:
country="malawi"
config=Config()
parameters = config.parameters(country)
country_dir = os.path.join(config.DIR_PATH, config.ANALYSES_DIR, country)
country_data_raw_dir = os.path.join(config.DATA_DIR,config.RAW_DIR,country)
country_data_processed_dir = os.path.join(config.DATA_DIR,config.PROCESSED_DIR,country)
country_data_exploration_dir = os.path.join(config.DATA_DIR,"exploration",country)
drought_data_exploration_dir= os.path.join(config.DATA_DIR, "exploration",  'drought')
cams_data_dir=os.path.join(drought_data_exploration_dir,"CAMS_OPI")
cams_tercile_path=os.path.join(cams_data_dir,"CAMS_tercile.nc")
chirps_monthly_dir=os.path.join(drought_data_exploration_dir,"CHIRPS")
chirps_monthly_path=os.path.join(chirps_monthly_dir,"chirps_global_monthly.nc")

In [5]:
chirpsgefs_dir = os.path.join(config.DROUGHTDATA_DIR,"chirps_gefs")

In [6]:
adm1_bound_path=os.path.join(country_data_raw_dir,config.SHAPEFILE_DIR,parameters["path_admin1_shp"])
adm2_bound_path=os.path.join(country_data_raw_dir,config.SHAPEFILE_DIR,parameters["path_admin2_shp"])

#### Rainy season
Compute a datetimeindex with all dates across all rainy seasons

In [7]:
#path to data start and end rainy season
df_rain=pd.read_csv(os.path.join(country_data_processed_dir,"dry_spells","rainy_seasons_detail_2000_2020_mean_back.csv"))
df_rain["onset_date"]=pd.to_datetime(df_rain["onset_date"])
df_rain["cessation_date"]=pd.to_datetime(df_rain["cessation_date"])

In [8]:
df_rain

,ID,pcode,ADM2_EN,season_approx,onset_date,onset_month,cessation_date,cessation_month,rainy_season_duration,rainy_season_rainfall
0,6.0,MW101,Chitipa,1999,NaT,NaN,2000-04-11,4.0,NaN,NaN
1,6.0,MW101,Chitipa,2000,2000-11-15,11.0,2001-04-03,4.0,140.0,1048.5
2,6.0,MW101,Chitipa,2001,2001-12-06,12.0,2002-04-21,4.0,137.0,1017.4
3,6.0,MW101,Chitipa,2002,2002-12-13,12.0,2003-04-12,4.0,121.0,903.0
4,6.0,MW101,Chitipa,2003,2003-12-02,12.0,2004-03-17,3.0,107.0,736.4
...,...,...,...,...,...,...,...,...,...,...
699,3.0,MW315,Blantyre City,2016,2016-11-23,11.0,2017-03-26,3.0,124.0,925.3
700,3.0,MW315,Blantyre City,2017,2017-11-13,11.0,2018-04-11,4.0,150.0,749.5
701,3.0,MW315,Blantyre City,2018,2018-11-24,11.0,2019-03-19,3.0,116.0,988.5
702,3.0,MW315,Blantyre City,2019,2019-11-24,11.0,2020-03-25,3.0,123.0,678.4


In [30]:
# #surprised 2020 earlies start is 01-12..
# df_rain[df_rain.onset_date.dt.year==2020]

In [10]:
#set the onset and cessation date for the seasons with them missing (meaning there was no dry spell data from start/till end of the season)
df_rain_filled=df_rain.copy()
df_rain_filled=df_rain_filled[(df_rain_filled.onset_date.notnull())|(df_rain_filled.cessation_date.notnull())]
df_rain_filled[df_rain_filled.onset_date.isnull()]=df_rain_filled[df_rain_filled.onset_date.isnull()].assign(onset_date=lambda df: pd.to_datetime(f"{df.season_approx.values[0]}-11-01"))
df_rain_filled[df_rain_filled.cessation_date.isnull()]=df_rain_filled[df_rain_filled.cessation_date.isnull()].assign(cessation_date=lambda df: pd.to_datetime(f"{df.season_approx.values[0]+1}-07-01"))

In [11]:
#get min onset and max cessation for each season across all admin2's
df_rain_seas=df_rain_filled.groupby("season_approx",as_index=False).agg({'onset_date': np.min,"cessation_date":np.max})

In [12]:
df_rain_seas

,season_approx,onset_date,cessation_date
0,1999,1999-11-01,2000-04-14
1,2000,2000-11-01,2001-04-20
2,2001,2001-11-17,2002-04-21
3,2002,2002-11-06,2003-04-26
4,2003,2003-11-07,2004-04-22
5,2004,2004-11-05,2005-04-16
6,2005,2005-11-08,2006-04-17
7,2006,2006-11-13,2007-05-01
8,2007,2007-11-01,2008-03-24
9,2008,2008-12-08,2009-04-06


In [13]:
all_dates=pd.Index([])
for i in df_rain_seas.season_approx.unique():
    seas_range=pd.date_range(df_rain_seas[df_rain_seas.season_approx==i].onset_date.values[0],df_rain_seas[df_rain_seas.season_approx==i].cessation_date.values[0])
    all_dates=all_dates.union(seas_range)

In [14]:
all_dates

Index([1999-11-01 00:00:00, 1999-11-02 00:00:00, 1999-11-03 00:00:00,
       1999-11-04 00:00:00, 1999-11-05 00:00:00, 1999-11-06 00:00:00,
       1999-11-07 00:00:00, 1999-11-08 00:00:00, 1999-11-09 00:00:00,
       1999-11-10 00:00:00,
       ...
       2021-06-22 00:00:00, 2021-06-23 00:00:00, 2021-06-24 00:00:00,
       2021-06-25 00:00:00, 2021-06-26 00:00:00, 2021-06-27 00:00:00,
       2021-06-28 00:00:00, 2021-06-29 00:00:00, 2021-06-30 00:00:00,
       2021-07-01 00:00:00],
      dtype='object', length=3639)

### Download CHIRPS-GEFS Africa data
We focus on the 15 day forecast, which is released every day. We define a dry spell occuring during that period if the cell with the maximum value within an admin2 is forecasted to receive less than 2mm of rainfall

We are focussing on the Africa data, since global data gets massive. Nevertheless, even for Africa it gets massive. 

In [15]:
#ftp url, where year and the start_date are variable
#start_date is the day of the year for which the forecast starts
chirpsgefs_ftp_url_africa_15day="https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.{year}.{start_day}.tif"

In [16]:
#NOPE, ACCORDING TO CHC SHOULDN"T USE THE 2020 data of that URL!! Just keeping to make sure I document it
# #ftp url, where year and the start_date are variable
# #start_date is the day of the year for which the forecast starts
# chirpsgefs_ftp_url_africa_15day_2020="https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip/15day/Africa/precip_mean/data.{year}.{start_day}.tif"

In [17]:
#part of 2020 data is missing. Might be available with this URL, but uncertain what the difference is. Mailed Pete Peterson on 02-03
#https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip/15day/Africa/precip_mean/

In [18]:
def download_chirpsgefs(date,output_dir,chirpsgefs_ftp_url,days=""):
    """
    Download the chirps-gefs africa 15 day forecast for the given year and day of the year
    Currently in expiremntation style code
    days: number of days forecast predicts ahead. When using 15 day, set to empty string
    """
    
    year=date.year
    day_year=str(date.timetuple().tm_yday).zfill(3)
    date_str=date.strftime("%Y%m%d")
    chirpsgefs_filepath = os.path.join(chirpsgefs_dir, f"chirpsgefs{days}_africa_{date_str}.tif")
    if not os.path.exists(chirpsgefs_filepath):
        print(date_str)
        print(chirpsgefs_ftp_url.format(year=year,start_day=day_year))
        try:
            download_ftp(chirpsgefs_ftp_url.format(year=year,start_day=day_year), chirpsgefs_filepath)
        except Exception as e: 
            print(f'CHIRPS-GEFS data not available for {date}')
            print(e)

In [21]:
# only needed if not downloaded yet
#download all the data
for d in all_dates:
    download_chirpsgefs(d,chirpsgefs_dir,chirpsgefs_ftp_url_africa_15day)

20200101
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2020.001.tif
CHIRPS-GEFS data not available for 2020-01-01 00:00:00
HTTP Error 404: Not Found
20200102
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2020.002.tif
CHIRPS-GEFS data not available for 2020-01-02 00:00:00
HTTP Error 404: Not Found
20200103
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2020.003.tif
CHIRPS-GEFS data not available for 2020-01-03 00:00:00
HTTP Error 404: Not Found
20200104
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2020.004.tif
CHIRPS-GEFS data not available for 2020-01-04 00:00:00
HTTP Error 404: Not Found
20200105
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2020.005.tif
CHIRPS-GEFS data not available 

In [19]:
#redownload broken files
for d in ["2010-05-07","2011-05-17","2012-02-29","2012-05-17","2013-05-17","2015-05-17","2016-02-29","2016-05-17","2017-05-17","2018-05-17","2019-05-17","2000-10-23","2002-09-09","2007-04-30"]:
    date=pd.to_datetime(d)
    download_chirpsgefs(date,chirpsgefs_dir)

20110517
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2011.137.tif
CHIRPS-GEFS data not available for 2011-05-17 00:00:00
HTTP Error 404: Not Found
20120229
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2012.060.tif
CHIRPS-GEFS data not available for 2012-02-29 00:00:00
HTTP Error 404: Not Found
20120517
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2012.138.tif
CHIRPS-GEFS data not available for 2012-05-17 00:00:00
HTTP Error 404: Not Found
20130517
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2013.137.tif
CHIRPS-GEFS data not available for 2013-05-17 00:00:00
HTTP Error 404: Not Found
20150517
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/15day/Africa/precip_mean/data.2015.137.tif
CHIRPS-GEFS data not available 

In [36]:
def ds_maxcell(ds,raster_transform,date,adm_path):
    #compute statistics on level in adm_path for all dates in ds
    df=gpd.read_file(adm_path)
    df["max_cell_touched"] = pd.DataFrame(
    zonal_stats(vectors=df, raster=ds.values, affine=raster_transform, all_touched=True,nodata=np.nan))["max"]
    df["min_cell_touched"] = pd.DataFrame(
    zonal_stats(vectors=df, raster=ds.values, affine=raster_transform, all_touched=True,nodata=np.nan))["min"]
    df["max_cell"] = pd.DataFrame(
    zonal_stats(vectors=df, raster=ds.values, affine=raster_transform, nodata=np.nan))["max"]
    df["min_cell"] = pd.DataFrame(
    zonal_stats(vectors=df, raster=ds.values, affine=raster_transform, nodata=np.nan))["min"]
    df["mean_cell"] = pd.DataFrame(
    zonal_stats(vectors=df, raster=ds.values, affine=raster_transform, nodata=np.nan))["mean"]
    df["date"]=pd.to_datetime(date)
    #define dryspell occuring if the cell touching the region with the maximum values is expected to receive less than 2mm of rain
#     df["dryspell"]=np.where(df["max_cell_touched"]<=2,1,0)
    df["date_forec_end"]=df["date"]+timedelta(days=14)
       
    return df

In [37]:
#only needed if new data added, else can load df_hist from file
#for each forecast date, compute the occurence of a dry spell and convert to a dataframe
df_list=[]
for filename in os.listdir(chirpsgefs_dir):
    if filename.endswith(".tif"):
        date=pd.to_datetime(re.split("[.\_]+", filename)[-2],format="%Y%m%d")
        if date in all_dates_2010:
            try:
                rds = rioxarray.open_rasterio(os.path.join(chirpsgefs_dir,filename))
                df_date=ds_maxcell(rds.sel(band=1),rds.rio.transform(),date,adm2_bound_path)
                df_list.append(df_date)
            except Exception as e:
                print(e)
                print(filename)
                print(date)
df_hist_all=pd.concat(df_list)

KeyboardInterrupt: 

In [25]:
#TODO: decide if wanna use same start and end of rainy season for all adm2's or do according to original data
#probs wanna do according to the adm2 data since there might be large differences
#which means we have to remove the dates currently in df_hist that are outside the rainy season for adm2s

In [ ]:
len(df_hist_all)

In [26]:
len(df_hist_all)

162272

In [27]:
# df_hist_all=df_hist.copy()

In [36]:
df_rain_filled

,ID,pcode,ADM2_EN,season_approx,onset_date,onset_month,cessation_date,cessation_month,rainy_season_duration,rainy_season_rainfall,region
0,6.0,MW101,Chitipa,1999,1999-11-01,NaN,2000-04-30,4.0,NaN,NaN,Northern
1,6.0,MW101,Chitipa,2000,2000-11-01,11.0,2001-06-09,6.0,221.0,3402.6,Northern
2,6.0,MW101,Chitipa,2001,2001-11-03,11.0,2002-05-16,5.0,195.0,3412.3,Northern
3,6.0,MW101,Chitipa,2002,2002-11-07,11.0,2003-05-21,5.0,196.0,2994.6,Northern
4,6.0,MW101,Chitipa,2003,2003-11-26,11.0,2004-05-23,5.0,180.0,3015.3,Northern
...,...,...,...,...,...,...,...,...,...,...,...
699,3.0,MW315,Blantyre City,2016,2016-11-20,11.0,2017-03-18,3.0,119.0,1144.6,Southern
700,3.0,MW315,Blantyre City,2017,2017-11-11,11.0,2018-03-17,3.0,127.0,902.8,Southern
701,3.0,MW315,Blantyre City,2018,2018-11-23,11.0,2019-04-03,4.0,132.0,1303.2,Southern
702,3.0,MW315,Blantyre City,2019,2019-11-23,11.0,2020-03-15,3.0,114.0,907.5,Southern


In [47]:
#remove the adm2-date entries outside the rainy season for that specific adm2
#before we included all forecasts within the min start of the rainy season and max end across the whole country
list_hist_rain_adm2=[]
for a in df_hist_all.ADM2_EN.unique():
    dates_adm2=pd.Index([])
    for i in df_rain_filled[df_rain_filled.ADM2_EN==a].season_approx.unique():
        seas_range=pd.date_range(df_rain_filled[(df_rain_filled.ADM2_EN==a)&(df_rain_filled.season_approx==i)].onset_date.values[0],df_rain_filled[(df_rain_filled.ADM2_EN==a)&(df_rain_filled.season_approx==i)].cessation_date.values[0])
        dates_adm2=dates_adm2.union(seas_range)
    list_hist_rain_adm2.append(df_hist_all[(df_hist_all.ADM2_EN==a)&(df_hist_all.date.isin(dates_adm2))])
df_hist_rain_adm2=pd.concat(list_hist_rain_adm2)

In [48]:
df_hist_rain_adm2

,Shape_Leng,Shape_Area,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry,max_cell_touched,min_cell_touched,max_cell,min_cell,date_forec_end
0,2.036817,0.179379,Balaka,MW312,None,None,None,Southern,MW3,Malawi,MW,2013-04-18,2018-10-16,None,"POLYGON ((35.20154 -14.75811, 35.20882 -14.768...",0.000000,0.000000,0.000000,0.000000,2013-05-03
0,2.036817,0.179379,Balaka,MW312,None,None,None,Southern,MW3,Malawi,MW,2016-11-25,2018-10-16,None,"POLYGON ((35.20154 -14.75811, 35.20882 -14.768...",86.308090,61.315823,86.308090,61.315823,2016-12-10
0,2.036817,0.179379,Balaka,MW312,None,None,None,Southern,MW3,Malawi,MW,2016-02-13,2018-10-16,None,"POLYGON ((35.20154 -14.75811, 35.20882 -14.768...",138.367172,89.876709,131.065704,89.876709,2016-02-28
0,2.036817,0.179379,Balaka,MW312,None,None,None,Southern,MW3,Malawi,MW,2016-02-07,2018-10-16,None,"POLYGON ((35.20154 -14.75811, 35.20882 -14.768...",72.439468,34.536011,66.885689,37.520149,2016-02-22
0,2.036817,0.179379,Balaka,MW312,None,None,None,Southern,MW3,Malawi,MW,2018-03-18,2018-10-16,None,"POLYGON ((35.20154 -14.75811, 35.20882 -14.768...",49.623730,22.383837,45.267921,22.383837,2018-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,0.295003,0.003375,Zomba City,MW314,None,None,None,Southern,MW3,Malawi,MW,2007-04-23,2018-10-16,None,"POLYGON ((35.35322 -15.35886, 35.35336 -15.358...",55.081516,24.832340,55.081516,55.081516,2007-05-08
31,0.295003,0.003375,Zomba City,MW314,None,None,None,Southern,MW3,Malawi,MW,2007-04-24,2018-10-16,None,"POLYGON ((35.35322 -15.35886, 35.35336 -15.358...",10.804195,3.401111,10.125187,10.125187,2007-05-09
31,0.295003,0.003375,Zomba City,MW314,None,None,None,Southern,MW3,Malawi,MW,2007-04-25,2018-10-16,None,"POLYGON ((35.35322 -15.35886, 35.35336 -15.358...",48.932186,24.592554,48.932186,48.932186,2007-05-10
31,0.295003,0.003375,Zomba City,MW314,None,None,None,Southern,MW3,Malawi,MW,2007-04-26,2018-10-16,None,"POLYGON ((35.35322 -15.35886, 35.35336 -15.358...",77.023117,56.614002,76.501030,76.501030,2007-05-11


In [50]:
df_hist_rain_adm2.date.dt.month.unique()

array([ 4, 11,  2,  3, 12,  1,  5,  6])

In [30]:
# #REMOVE: only for testing
# df_hist=df_hist_rain_adm2.copy()

In [51]:
#computation of df_hist can take rather long, so also save it and load it if no new info is added
hist_path=os.path.join(country_data_exploration_dir,"chirpsgefs","mwi_chirpsgefs_rainyseas_maxcell_test.csv")
# df_hist_sel=df_hist_rain_adm2[["date","date_forec_end","ADM1_EN","ADM2_EN","max_cell_touched","min_cell_touched","max_cell","min_cell"]]
# df_hist_sel.to_csv(hist_path,index=False)
df_hist_rain_adm2.drop("geometry",axis=1).to_csv(hist_path,index=False)
df_hist=pd.read_csv(hist_path)
df_hist["date"]=pd.to_datetime(df_hist["date"])
df_hist["date_forec_end"]=pd.to_datetime(df_hist["date_forec_end"])

In [ ]:
df_hist

In [ ]:
df_hist[df_hist.dryspell==1]

In [ ]:
#that is a huge number of dry spells... Expecting they mainly occur in the last months of the rainy season (May/June) but should check thatd
sns.displot(df_hist,x="max_cell_touched")

In [ ]:
#check how distribution changes when using min cell touched instead of max --> even more sensitive
sns.displot(df_hist,x="min_cell_touched")

In [ ]:
print("number of adm2-date combination with min cell touched <=2mm",len(df_hist[df_hist.min_cell_touched<=2]))

In [ ]:
print("number of adm2-date combination with max cell touched <=2mm",len(df_hist[df_hist.max_cell_touched<=2]))

In [ ]:
print("number of dates with at least one adm2 with max cell touched <=2mm",len(df_hist[df_hist.max_cell_touched<=2].date.unique()))

In [ ]:
# print("number of dates with at least one adm2 with max cell touched <=2mm",len(df_hist[df_hist.max_cell<=2].date.unique()))

### Load ds with dates within rainy seas

In [16]:
ds_list=[]
for d in [rainy_date for rainy_date in all_dates if rainy_date.year>=2010]:
    d_str=pd.to_datetime(d).strftime("%Y%m%d")
    filename=f"chirpsgefs_africa_{d_str}.tif"
    try:
        rds=rioxarray.open_rasterio(os.path.join(chirpsgefs_dir,filename))
        rds=rds.assign_coords({"time":pd.to_datetime(d)})
        rds=rds.sel(band=1)
        ds_list.append(rds)
    except Exception as e:
            print(e)
            print(filename)
            print(d_str)

/Volumes/GoogleDrive/Shared drives/Predictive Analytics/CERF Anticipatory Action/General - All AA projects/Data/raw/drought/chirps_gefs/chirpsgefs_africa_20100517.tif: No such file or directory
chirpsgefs_africa_20100517.tif
20100517
/Volumes/GoogleDrive/Shared drives/Predictive Analytics/CERF Anticipatory Action/General - All AA projects/Data/raw/drought/chirps_gefs/chirpsgefs_africa_20110517.tif: No such file or directory
chirpsgefs_africa_20110517.tif
20110517
/Volumes/GoogleDrive/Shared drives/Predictive Analytics/CERF Anticipatory Action/General - All AA projects/Data/raw/drought/chirps_gefs/chirpsgefs_africa_20120229.tif: No such file or directory
chirpsgefs_africa_20120229.tif
20120229
/Volumes/GoogleDrive/Shared drives/Predictive Analytics/CERF Anticipatory Action/General - All AA projects/Data/raw/drought/chirps_gefs/chirpsgefs_africa_20120517.tif: No such file or directory
chirpsgefs_africa_20120517.tif
20120517
/Volumes/GoogleDrive/Shared drives/Predictive Analytics/CERF Ant

In [17]:
ds_drys=xr.concat(ds_list,dim="time")

In [ ]:
ds_drys=ds_drys.sortby("time")

In [ ]:
ds_drys.to_netcdf(country_data_processed_dir,"chirps_gefs","mwi_chirpsgefs_15day.nc")

In [ ]:
ds_drys

### CHIRPS GEFS 5 day

In [8]:
df_ds=pd.read_csv(os.path.join(country_data_processed_dir,"dry_spells","dry_spells_during_rainy_season_list_2000_2020_mean.csv"))
df_ds["dry_spell_first_date"]=pd.to_datetime(df_ds["dry_spell_first_date"])
df_ds["dry_spell_last_date"]=pd.to_datetime(df_ds["dry_spell_last_date"])

In [17]:
#ftp url, where year and the start_date are variable
#start_date is the day of the year for which the forecast starts
chirpsgefs_ftp_url_africa_5day="https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/05day/Africa/precip_mean/data.{year}.{start_day}.tif"

In [18]:
# only needed if not downloaded yet
#download all the data
for d in df_ds.dry_spell_first_date.unique():
    download_chirpsgefs(pd.to_datetime(d),chirpsgefs_dir,chirpsgefs_ftp_url_africa_5day,days="_5day")

20090407
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/05day/Africa/precip_mean/data.2009.097.tif
20020419
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/05day/Africa/precip_mean/data.2002.109.tif
20090406
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/05day/Africa/precip_mean/data.2009.096.tif
20060417
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/05day/Africa/precip_mean/data.2006.107.tif
20110411
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/05day/Africa/precip_mean/data.2011.101.tif
20040316
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/05day/Africa/precip_mean/data.2004.076.tif
20030406
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/05day/Africa/precip_mean/data.2003.096.tif
20090405
https://data.chc.ucsb.edu/products/EWX/data/forecasts/CHIRPS-GEFS_precip_v12/05day/Afric

In [109]:
ds_list=[]
for d in df_ds.dry_spell_first_date.unique():
    d_str=pd.to_datetime(d).strftime("%Y%m%d")
    filename=f"chirpsgefs_5day_africa_{d_str}.tif"
    rds=rioxarray.open_rasterio(os.path.join(chirpsgefs_dir,filename))
    rds=rds.assign_coords({"time":pd.to_datetime(d)})
    rds=rds.sel(band=1)
    ds_list.append(rds)

In [110]:
ds_drys=xr.concat(ds_list,dim="time")

In [111]:
ds_drys=ds_drys.sortby("time")